![ss](images/zookeeper.png)

## zookeeper工作原理
###  ZOOKEEPER集群做这么一件事,保证ZOOKEEPER对外提供统一的信息, 解决集群中所有结点数据不统一的问题!!

    NN 高可用场景:
    ------------------------------------------------------------------------------------------------>t
      (NN1 通知zk1为是主NN)    (NN2 通知zk3为是主NN)   (zk1,zk2,zk3 同意NN1主,因为投票结果是(1,1,0),更新同步zkid=1)
    ------------------------------------------------------------------------------------------------>t
        (zk1征求所有zk的意见:缓存自己的zkid=1)   (zk2征求主zk的意见,缓存自己的zkid=1))       (zk1,zk2,zk3 拒绝zkid=1,NN2主,因为(0,0,1))
  
  
  NN1,NN2
  zk1,zk2,zk3
  


  ### 1.对外提供服务
    1.客户端相zookper注册(任意的节点),例如插入一个节点,非主节点要相主节点发起通知,主节点广播给所有的节点:
        A)汇集所有节点的"意见"(新的事物事物编号是不是线性增长的,是同意,不是,反对),反馈给其他从节点
        B)如果"同意",通知节点执行,并且所有节点保持相同结果(MESSAGE QUEUE)
    2.监听客户端.
    3.回调客户端的回调函数!
  ### 2.对内集群管理
  
      主节点选举的依据:
          A)事物ID,zxid,线性增长
          B)服务ID,severid
          zxid>severid
       当某个主节点挂掉后,从节点通过"过半选举"的方式,快速选举出新的主节点,zxid大的,severid大的从节点会被选为主节点!
  ### 2.zookeeper角色:
      leader,follower,observer
          follower和observer的区别是follow参与进选,observer不参与
 

## ZOOKER 集群搭建

    conf/zoo.cfg文件:
        tickTime=2000   //心跳 基本单位=2s
        dataDir=/Users/zdandljb/zookeeper/data   //数据目录下要指定myid
        dataLogDir=/Users/zdandljb/zookeeper/dataLog  
        clientPort=2181                  //客户端通信端口
        initLimit=5                      //从节点在initLimit*tickTime=10s 不和主保持心跳通信,主 认为 从 当机..
        syncLimit=2                      //主与从发送消息,相应时长 不能超过syncLimit*tickTime=4s
        server.1=server1:2888:3888       //2888是主从  通信端口
        server.2=server2:2888:3888       //3888是选举  通信端口
        server.3=server3:2888:3888
##  基本命令:
    create /myzxk zxkdata     //创建持久化节点
    create -e /myzxk zxkdata  //创建临时节点,czxid
    set /myzxk zxk            //更新节点,更新后观察,mzxid
    create /myzxk/home zxkhose //观察pzxid
    
    create -s /myzxk zxkdata  //创建序列 节点
    get /myzxk
    
    持久化(persistent) 临时(emphemeral)节点
    持久化:重启后还存在,保存在文件上面
    临时:随客户端 session失效而失效